In [16]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model_name="llama3-8b-8192",
    temperature=0,
    groq_api_key="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",
)


In [17]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/information-security-analyst-grc-itc/job/R-61563")
page_data = loader.load().pop().page_content

In [18]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """The following text is scraped from a company's careers page:

    {page_data}

    Your task is to extract all job postings and return each as a JSON object
    with the following keys only:
    - "role": The job title or role
    - "experience": Required or preferred experience
    - "skills": A list of required or preferred skills
    """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})

In [35]:
import chromadb
import json

client = chromadb.PersistentClient()
collection = client.get_or_create_collection(name="tech-stack")

tech_languages = [
    {"id": "1", "name": "Python", "description": "High-level, interpreted, general-purpose programming language."},
    {"id": "2", "name": "JavaScript", "description": "Versatile language primarily used for web development."},
    {"id": "3", "name": "Java", "description": "Object-oriented programming language used for web, Android, and enterprise development."},
    {"id": "4", "name": "C++", "description": "Compiled, high-performance language with object-oriented features."},
    {"id": "5", "name": "Go", "description": "Statically typed language designed for simplicity and performance."},
    {"id": "6", "name": "Rust", "description": "Memory-safe, high-performance language without garbage collection."},
    {"id": "7", "name": "Kotlin", "description": "Modern programming language for Android and JVM development."},
    {"id": "8", "name": "TypeScript", "description": "Typed superset of JavaScript that compiles to plain JavaScript."},
    {"id": "9", "name": "Ruby", "description": "Dynamic, interpreted language known for simplicity and productivity."},
    {"id": "10", "name": "Swift", "description": "Modern language developed by Apple for iOS and macOS development."}
]

json_output = json.dumps(tech_languages, indent=2)

# Print the JSON-formatted string
print(json_output)




[
  {
    "id": "1",
    "name": "Python",
    "description": "High-level, interpreted, general-purpose programming language."
  },
  {
    "id": "2",
    "name": "JavaScript",
    "description": "Versatile language primarily used for web development."
  },
  {
    "id": "3",
    "name": "Java",
    "description": "Object-oriented programming language used for web, Android, and enterprise development."
  },
  {
    "id": "4",
    "name": "C++",
    "description": "Compiled, high-performance language with object-oriented features."
  },
  {
    "id": "5",
    "name": "Go",
    "description": "Statically typed language designed for simplicity and performance."
  },
  {
    "id": "6",
    "name": "Rust",
    "description": "Memory-safe, high-performance language without garbage collection."
  },
  {
    "id": "7",
    "name": "Kotlin",
    "description": "Modern programming language for Android and JVM development."
  },
  {
    "id": "8",
    "name": "TypeScript",
    "description": "Typ

In [22]:
collection.add(
    ids=[item['id'] for item in tech_languages],
    documents=[item['name'] for item in tech_languages],
    metadatas=[{"description": item['description']} for item in tech_languages]
)
docs = collection.get()
docs

{'ids': ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'],
 'embeddings': None,
 'documents': ['Python',
  'JavaScript',
  'Java',
  'C++',
  'Go',
  'Rust',
  'Kotlin',
  'TypeScript',
  'Ruby',
  'Swift'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [{'description': 'High-level, interpreted, general-purpose programming language.'},
  {'description': 'Versatile language primarily used for web development.'},
  {'description': 'Object-oriented programming language used for web, Android, and enterprise development.'},
  {'description': 'Compiled, high-performance language with object-oriented features.'},
  {'description': 'Statically typed language designed for simplicity and performance.'},
  {'description': 'Memory-safe, high-performance language without garbage collection.'},
  {'description': 'Modern programming language for Android and JVM development.'},
  {'description': 'Typed superset of JavaScript that compiles to plain JavaScript.'}

In [32]:
res = collection.query(
    query_texts=['I need a python'],n_results=2
)
res['metadatas']

[[{'description': 'High-level, interpreted, general-purpose programming language.'},
  {'description': 'Dynamic, interpreted language known for simplicity and productivity.'}]]